Download Pytorch BigGraph pretrained embedding

In [ ]:
!wget https://dl.fbaipublicfiles.com/torchbiggraph/wikidata_translation_v1.tsv.gz -O wikidata_translation_v1.tsv.gz


--2024-10-16 16:54:05--  https://dl.fbaipublicfiles.com/torchbiggraph/wikidata_translation_v1.tsv.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.108, 3.163.189.51, 3.163.189.96, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38830884950 (36G) [application/gzip]
Saving to: ‘wikidata_translation_v1.tsv.gz’

wikidata_translatio 100%[===================>]  36.16G  41.4MB/s    in 13m 38s 

2024-10-16 17:07:43 (45.3 MB/s) - ‘wikidata_translation_v1.tsv.gz’ saved [38830884950/38830884950]



Query knowledge graph embedding for authors

In [ ]:
import pickle
import numpy as np
from smart_open import open
import tqdm


In [ ]:
# load mapping from author to Wikidata ID
with open('/content/author_qid_mapping (2).pickle', 'rb') as f:
    author2qid = pickle.load(f)

In [ ]:
# reverse dict
qid2author = {qid: author for author, qid in author2qid.items()}

In [ ]:
# query knowledge graph embedding for authors
from tqdm import tqdm
import numpy as np
import pickle

def map_authors_to_embeddings(graph_embedding_file, qid2author, out_file):
    author2embedding = {}

    # Open the graph embedding file
    with open(graph_embedding_file, encoding='utf-8') as fp:
        total_lines = sum(1 for line in fp)  # Get total lines for progress bar
        fp.seek(0)  # Reset file pointer after counting lines

        # Use tqdm to show a progress bar
        for line in tqdm(fp, total=total_lines, desc="Processing embeddings"):
            cols = line.split('\t')  # Split each line into columns

            entity_id = cols[0]

            # Check if the entity is a valid Wikidata QID
            if entity_id.startswith('<http://www.wikidata.org/entity/Q') and entity_id.endswith('>'):
                entity_id = entity_id.replace('<http://www.wikidata.org/entity/', '').replace('>', '')  # Extract QID

                # Check if the entity is in the selected entities and map it to the author
                if entity_id in qid2author:
                    author = qid2author[entity_id]
                    author2embedding[author] = np.array(cols[1:]).astype(np.float64)

    # Save the author-to-embedding dictionary to a pickle file
    with open(out_file, 'wb') as f:
        pickle.dump(author2embedding, f)

    print(f'Saved author-embedding mappings to {out_file}')


In [ ]:
map_authors_to_embeddings('/content/wikidata_translation_v1.tsv.gz', qid2author, 'author2embedding.pickle')

Processing embeddings: 100%|██████████| 78413186/78413186 [36:55<00:00, 35398.37it/s]


Saved author-embedding mappings to author2embedding.pickle


Check

In [ ]:
with open('/content/author2embedding.pickle', 'rb') as f:
    author2embedding = pickle.load(f)

In [ ]:
len(author2embedding)

4302